In [ ]:
import pandas as ps
import numpy as np
import cv2     # for capturing videos
import math   
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import shutil
from skvideo import io

# New version for video processing

In [ ]:
video_dir = "cleaned_data_videos/cropped_data_auto"
out_image_dir = "cleaned_data_images"
CROSS_VAL_DIR = "cross_validation_data"

In [ ]:
files = os.listdir(video_dir)
labs = [f[:3] for f in files]
print(np.unique(labs, return_counts=True))

### Make directories

In [ ]:
# make directories:
if not os.path.exists(out_image_dir):
    os.makedirs(out_image_dir)
if not os.path.exists(CROSS_VAL_DIR):
    os.makedirs(CROSS_VAL_DIR)   
for mod in ["covid", "pneumonia", "regular"]: # viral
    if not os.path.exists(os.path.join(out_image_dir, mod)):
        os.makedirs(os.path.join(out_image_dir, mod))

## Videos to images - automatically select every x frame

Takes all videos from video_dir and writes the frames to out_image_dir

### prepare script

In [ ]:
TAKE_CLASSES = ["covid", "pneumonia", "regular"]
TAKE_MODE = ["Linear", "Convex"]
FRAMERATE = 3 # saves automatically 2 frames per second
MAX_FRAMES = 30
POCUS_IMAGE_DIR = "../../data/pocus_images"
POCUS_VIDEO_DIR = "../../data/pocus_videos"
out_image_dir = "../../data/image_dataset"
if not os.path.exists(out_image_dir):
    os.makedirs(out_image_dir)  
for mod in TAKE_CLASSES:
    if not os.path.exists(os.path.join(out_image_dir, mod)):
        os.makedirs(os.path.join(out_image_dir, mod))

In [ ]:
def label_to_dir(lab):
    if lab == "Cov":
        label = "covid"
    elif lab == "Pne" or lab == "pne":
        label = "pneumonia"
    elif lab == "Reg":
        label = "regular"
    elif lab == "Vir":
        label = "viral"
    else:
        raise ValueError("Wrong label! " + lab)
    return label # os.path.join(label, lab + "-")

In [ ]:
# copy all images from pocus_images
for mode in TAKE_MODE:
    for fp in os.listdir(os.path.join(POCUS_IMAGE_DIR, mode)):
        if fp[-3:] in ["png", "jpg", "jpeg"]:
            label_dir = label_to_dir(fp[:3])
            if label_dir in TAKE_CLASSES:
                # print(os.path.join(POCUS_IMAGE_DIR, mode,fp), os.path.join(out_image_dir, label_dir))
                shutil.copy(os.path.join(POCUS_IMAGE_DIR, mode,fp), os.path.join(out_image_dir, label_dir))

In [ ]:
for mode in TAKE_MODE:
    vid_files = os.listdir(os.path.join(POCUS_VIDEO_DIR, mode))
    for i in range(len(vid_files)):

        # skip non video files
        if vid_files[i][-3:].lower() not in ["mpeg", "gif", "mp4", "m4v", "avi", "mov"]:
            continue

        # define video path
        video_path = os.path.join(POCUS_VIDEO_DIR, mode, vid_files[i])
        # determine label
        label = label_to_dir(vid_files[i][:3])
        # determine out path based on label
        out_path = os.path.join(out_image_dir, label)

        # read and write if video
        cap = cv2.VideoCapture(video_path)   # capturing the video from the given path
        frameRate = cap.get(5) #frame rate
        # num_frames = cap.get(7)
        every_x_image = int(frameRate/FRAMERATE)
        print(vid_files[i], "framerate", cap.get(5),"width", cap.get(3), "height", cap.get(4), "number frames:", cap.get(7))
        print("--> taking every ", every_x_image, "th image")
        x=1
        nr_selected = 0
        while cap.isOpened() and nr_selected<MAX_FRAMES:
            frameId = cap.get(1) #current frame number
            ret, frame = cap.read()
            if (ret != True):
                break
            if (frameId % every_x_image == 0):
                # storing the frames in a new folder named test_1
                filename = os.path.join(out_path, vid_files[i] +"_frame%d.jpg" % frameId)
                cv2.imwrite(filename, frame)
                nr_selected += 1
                print(filename)
        cap.release()

## Previous version of video to images (no script)

In [ ]:
# SET FRAMERATE
FRAMERATE = 3 # saves automatically 2 frames per second
MAX_FRAMES = 30

In [ ]:
vid_files = os.listdir(video_dir)
for i in range(len(vid_files)):
    
    # skip unnecessary files
    if vid_files[i][0]==".":
        # DStore etc
        continue
    
    # define video path
    # print("video", vid_files[i], "number ",i, "out of ", len(vid_files))
    video_path = os.path.join(video_dir, vid_files[i])
    
    # determine label
    if vid_files[i][:3]=="Cov":
        label = "covid"
    elif vid_files[i][:3]=="Pne" or vid_files[i][:3]=="pne":
        label = "pneumonia"
    elif vid_files[i][:3]=="Reg":
        label = "regular"
    else:
        raise ValueError("Wrong label! "+ vid_files[i])
    out_path = os.path.join(out_image_dir, label)
    
    # copy if image
    if vid_files[i][-3:]=="jpg" or vid_files[i][-3:]=="png":
        shutil.copy(video_path, out_path)
        continue
    
    # read and write if video
    cap = cv2.VideoCapture(video_path)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    # num_frames = cap.get(7)
    every_x_image = int(frameRate/FRAMERATE)
    print(vid_files[i], "framerate", cap.get(5),"width", cap.get(3), "height", cap.get(4), "number frames:", cap.get(7))
    print("--> taking every ", every_x_image, "th image")
    x=1
    nr_selected = 0
    while cap.isOpened() and nr_selected<MAX_FRAMES:
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % every_x_image == 0):
            # storing the frames in a new folder named test_1
            filename = os.path.join(out_path, vid_files[i] +"_frame%d.jpg" % frameId)
            cv2.imwrite(filename, frame)
            nr_selected += 1
            print(filename)
    cap.release()

# Script to select frames manually from one video

In [ ]:
# define video path (ONE VIDEO)
video_path = "cleaned_data_videos/cropped_data_manual/Reg-Butterfly.mp4"
# define where to save outputs
out_image_dir = "cleaned_data_images"
# how many frames do you want to check?
check_every = 2 # displays every second frame
out_name = "Reg-Butterfly.mp4"

In [ ]:
# make subdirectories if they don't exist
for mod in ["covid", "pneunomia", "regular"]:
    if not os.path.exists(os.path.join(out_image_dir, mod)):
        os.makedirs(os.path.join(out_image_dir, mod))

# out_name = "Cov-MSU-COVID Lung 2-Blines.mp4"

# determine label
if out_name[:3]=="Cov":
    label = "covid"
elif out_name[:3]=="Pne" or vid_files[i][:3]=="pne":
    label = "pneunomia"
elif out_name[:3]=="Reg":
    label = "regular"
else:
    raise ValueError("Wrong label! "+ vid_files[i])
# Final outpath: out directory plus the label
out_path = os.path.join(out_image_dir, label)

# read and write if frame is selected
cap = cv2.VideoCapture(video_path)   # capturing the video from the given path
decision=0
n_frames = cap.get(7)
selected_frames = []
while cap.isOpened() and decision!=2:
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    # only display every x frames (more diverse, not as much work)
    if frameId>46 and frameId%check_every==0:
        print("already selected ", len(selected_frames), "frames")
        if len(selected_frames)>0:
             print("last selected was frame", selected_frames[-1])
        # display image
        plt.imshow(frame)
        plt.title("Frame "+str(frameId)+" out of "+str(n_frames))
        plt.show()
        decision = int(input("Take image? (No:0, Yes:1, Stop:2)  "))
        if decision==1:
            # storing the frames in a new folder named test_1
            filename = os.path.join(out_path, out_name +"_frame%d.jpg" % frameId)
            cv2.imwrite(filename, frame)
            selected_frames.append(frameId)
cap.release()

In [ ]:
fp = "/Users/ninawiedemann/Desktop/Projects/covid19_pocus_ultrasound.nosync/data/pocus/cleaned_data_images_backup/covid/test"
img = cv.

## Automatically process videos

In [ ]:
# for cv2:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter(
    os.path.join(base_path, "test.avi"), fourcc,
    20.0, tuple(curr_size)
)
for x in vid_arr:
    writer.write(x.astype("uint8"))
writer.release()

In [ ]:
base_path = "../../../data_pocovid/DATASET/bolzano.nosync/"
for vid in os.listdir(base_path):
    if vid != "VID-20200427-WA0039.mp4": # .startswith("."):
        continue
    # vid = "Linee B 9 top.mp4"
    # print(os.listdir(base_path))
    # print(os.path.exists(os.path.join(base_path, vid)))
    cap = cv2.VideoCapture(os.path.join(base_path, vid))
    arr = []
    counter = 0
    while True:
        ret, frame = cap.read()
        counter += 1
        if not ret:
            break
        if counter <2:
            plt.figure(figsize=(10,10))
            plt.imshow(frame[120:420, 40:340]) # [90:390, 15:315]) # [130:480, 0:350]) # [40:240, 50:250])
            plt.show()

        arr.append(frame[120:420, 40:340]) # [200:1000, 750:1550])
    vid_arr = np.asarray(arr)
    print(vid_arr.shape)
    # curr_size = vid_arr.shape[1:3]
    # print("output video size", curr_size)
    io.vwrite(os.path.join(base_path, "cropped", "Cov_bolzano_"+vid.split(".")[0]+".mpeg"), vid_arr, outputdict={"-vcodec":"mpeg2video"})
    cap.release()

## Relevant for filling data table: Get framerate of videos

In [ ]:
base_path = "/Users/ninawiedemann/Desktop/Projects/data_pocovid/DATASET/bolzano.nosync/cropped"
f_list = os.listdir(base_path)
f_list = sorted([vid for vid in f_list if not vid.startswith(".")]) #  and "avi" in vid or "jpg" in vid])
f_list

In [ ]:
for vid in f_list:
    # if not vid.startswith(".") and "avi" in vid or "jpg" in vid:
    # print(vid)
    # # Code to rename files
    # new_fn = vid.replace(",", "")
    # new_fn = new_fn.replace(" ", "-")
    # shutil.move(os.path.join(base_path, vid), os.path.join(base_path, new_fn))
    cap = cv2.VideoCapture(os.path.join(base_path, vid))
    print(str(int(cap.get(3)))+"x"+str(int(cap.get(4))))
    # print(vid)
    # print(int(cap.get(5)))
    if "avi" in vid:
        # print("pocus_videos/convex")
        pass # print("video")
    else:
        pass # print("image")
        # print("pocus_images/convex")
    cap.release()

In [ ]:
base_path = "../../../data_pocovid/new_git_issues.nosync/combatting_article"
f_list = os.listdir("../../../data_pocovid/new_git_issues.nosync/combatting_article")

In [ ]:
data_table = pd.read_csv("../../data/pocovid_data.csv")
butterfly = data_table["Filename"].values
butterfly = [b for b in butterfly if "utterfly" in b]

In [ ]:
import json
with open("../../data/data_from_butterfly.json", "w") as outfile:
    json.dump(butterfly, outfile)

# Images to video

In [ ]:
from skvideo import io

In [ ]:
path = "cleaned_data_videos/new_Marion/covid"
fn = "Cov-Butterfly-Irregular Pleura with Multip.mp4" # _frame0.jpg"
out_path = "cleaned_data_videos/cropped_data_manual_cleaned"


In [ ]:
file_list = []
for frame in os.listdir(path):
    if frame.startswith(fn):
        file_list.append(os.path.join(path, frame))

In [ ]:
# Define the codec and create VideoWriter object

arr = []
for frame in file_list:
    # ret, frame = cap.read()
    arr.append(cv2.imread(frame))
arr = np.array(arr)
# Release everything if job is finished
# cv2.destroyAllWindows()
arr.shape

In [ ]:
io.vwrite(os.path.join(out_path, fn[:-4]+".mpeg"), arr, outputdict={"-vcodec":"mpeg2video"})

# Video dataset - Previous version

In [ ]:
from tqdm import tqdm

In [ ]:
import os
path='enter your path\'

#Train
videos_train=os.listdir(path+'train\\')
train = pd.DataFrame()
train['video_name'] = videos_train
train = train[:-1]

#Test
videos_test=os.listdir(path+'test\\')
test = pd.DataFrame()
test['video_name'] = videos_test
test = test[:-1]


In [ ]:
# storing the frames from training videos
fn = "cleaned_data_videos/cropped_data_auto/Cov-clarius3.mp4"
count = 0
cap = cv2.VideoCapture(fn)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
num = cap.get(7)
x=1
print(frameRate, num)

cap.release()
    


In [ ]:
25 * 12

In [ ]:
# storing the frames from testing videos
for i in tqdm(range(test.shape[0])):
    count = 0
    videoFile = test['video_name'][i]
    cap = cv2.VideoCapture(path+'test\\'+videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named test_1
            filename =path+'test_1/' + videoFile +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

## 10 fold cross validation

In [ ]:
import os
os.listdir("train")

In [ ]:
os.listdir("test")

### Compute all framerates etc:

In [ ]:
csv = pd.read_csv("../../dataset_metadata.csv")

In [ ]:
filenames = csv["Filename"].values
locations = csv["Current location"].values
prev_fr = csv["Length (frames)"].values

In [ ]:
base="../../data"

In [ ]:
for fn, location, prev in zip(filenames, locations, prev_fr):
    # print(fn, location)
    location = location.lower()
    if location.startswith("data/"):
        location = location[5:]
        
    if fn[-4:] in [".png", ".jpg", "jpeg", ".JPG"]:
        print(1)
    else:
        if not os.path.exists(os.path.join(base, location, fn)):
            # print("ERROR does not exist", fn)
            print(prev)
        else:
            cap = cv2.VideoCapture(os.path.join(base, location, fn))
            print(round(cap.get(7)))
            # print(str(int(cap.get(3)))+"x"+str(int(cap.get(4))))